In [ ]:
!pip install afs2-model
!pip install flask

In [ ]:
from afs import models
import os, numpy, pandas, datetime, time, json, requests, re, configparser
from sklearn.preprocessing import MinMaxScaler
from flask import Flask, jsonify
from flask import request
import json
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import os

def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
    return numpy.array(dataX)

def load_rnn_model(filepath, dataset, count=1):
    print('load rnn')
    model = load_model(filepath)
    raw_data = dataset
    print('raw_data: {}'.format(raw_data))
    testPredict = model.predict(dataset)
    return raw_data, testPredict

afs_models = models()
app = Flask(__name__)
# downland model
@app.route("/model", methods=['POST'])
def model():
    data=request.get_json()
    save_path = data['save_model_to']
    model_repository_name = data['name']
    model_name = data['version']
    save_path = os.path.join(save_path,model_repository_name)
    afs_models.download_model(save_path=save_path, model_repository_name=model_repository_name, model_name=model_name)
    return jsonify({'status': 'success','model':save_path})

@app.route("/predict", methods=['POST'])
def predict():
    data_list1 = []
    data_list2 = []
    data_list3 = []
    data_list_all = []
    post_data1 = request.form.get('feature_1')
    post_data2 = request.form.get('feature_2')
    for value in post_data1.split(','):
        data_list1.append(float(value))
    for value in post_data2.split(','):
        data_list2.append(float(value))
    data_list3.append(data_list1)
    data_list3.append(data_list2)
    data_list_all.append(data_list3)
    data = numpy.array(data_list_all)
    raw_data, predict_data = load_rnn_model('/rnn_model.h5', data)
    for datas in predict_data:
        for data in datas:
            predict_data = data
    return jsonify({'message': str(predict_data)})

@app.route("/check_model", methods=['POST'])
def check_model():
    data = request.get_json()
    filepath = data['filepath']
    if os.path.isfile(filepath):
        print("EXISTS")
        return jsonify({'status': 'EXISTS','model':filepath})
    else:
        print("NOT EXISTS")
        return jsonify({'status': 'NOT EXISTS','model':filepath})
    
import os
if __name__ == "__main__":
    port = int(os.environ.get('cloud_inference_port', 5000))
     #port = int('7500')
    app.run(host='0.0.0.0', port=port)